<a href="https://colab.research.google.com/github/AlejandroVillazonG/Tareas_INF395/blob/main/T2/4_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src="https://www.inf.utfsm.cl/images/slides/Departamento-de-Informtica_HORIZONTAL.png" title="Title text" width= 800 /></center>
<hr style="height:2px;border:none"/>
<H1 align='center'> DESAFÍO TAREA 2</H1>

<H3 align='center'> INF395 2023-2 </H3>
<hr style="height:2px;border:none"/>

## Integrantes:
* Joaquín Aguirre (201910031-9)
* Alejandro Villazón (201910009-2)
* Dominique Yessouroun (201910005-K)

Importamos las librerías necesarias:

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import os
import json
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50

import warnings
# Ignorar las advertencias
warnings.filterwarnings("ignore")

Definimos funciones auxiliares:

In [2]:
def preprocess_image(main_path, path_image, image_size):
    image = keras.utils.load_img(main_path + path_image)
    image = image.resize((image_size, image_size))
    return keras.utils.img_to_array(image, dtype='uint8')

def preprocess_train_test_set(main_path, paths, image_size):
    return np.array([preprocess_image(main_path, path_image, image_size) for path_image in paths])

In [3]:
def create_macro_f1(y_true, y_pred, num_classes):
    """
    Calcula el Macro F1-Score.
    """
    y_pred = tf.one_hot(tf.argmax(y_pred, axis=1), depth=num_classes)

    tp = tf.cast(tf.math.count_nonzero(y_pred * y_true, axis=1), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1-y_true), axis=1), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1-y_pred) * y_true, axis=1), tf.float32)

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    f1 = 2*precision*recall / (precision + recall)
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)

    return tf.reduce_mean(f1)

def transform_dishes(original_dishes, mapping_dishes, category_mapping):
    transformed_dishes = {}
    for original_key, dish in original_dishes.items():
        for category_key, category_value in category_mapping.items():
            if dish in mapping_dishes[category_value].values():
                # Encontrar la clave interna dentro del diccionario de categoría
                internal_key = [int(k) for k, v in mapping_dishes[category_value].items() if v == dish][0]
                # Convertir la clave original a int para guardarla en el resultado
                original_key_int = int(original_key)
                # Asegurar que la categoría exista en el diccionario resultante
                if category_key not in transformed_dishes:
                    transformed_dishes[category_key] = {}
                # Agregar el mapeo al diccionario resultante
                transformed_dishes[category_key][original_key_int] = internal_key
    return transformed_dishes

def create_model(image_size, num_classes):
  cnn_model = ResNet50(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
  for layer in cnn_model.layers:
      layer.trainable = False
  model = keras.Sequential([
      cnn_model,
      layers.Conv2D(512, 2, activation='relu'),
      layers.Flatten(),
      layers.Dense(128, activation='relu'),
      layers.Dropout(0.5),
      layers.Dense(num_classes, activation='softmax')
  ])
  def macro_f1(y_true, y_pred):
    return create_macro_f1(y_true, y_pred, num_classes)

  model.compile(optimizer=keras.optimizers.AdamW(),
                loss=['categorical_crossentropy'],
                metrics=[macro_f1])
  return model

In [4]:
E_S = keras.callbacks.EarlyStopping(monitor='val_macro_f1', mode='max',
                                    patience=10, restore_best_weights=True)

Nos conectamos al drive:

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
path = '/content/drive/MyDrive/INF395/T2/'

## Preprocesar imágenes

In [7]:
!unzip '{path}inf-395-tarea-2.zip' > /dev/null

In [8]:
main_path = '/content/'

Definimos el tamaño que tendran las imágenes.

In [9]:
image_size = 225

In [10]:
df_test = pd.read_csv(main_path+'test.csv')

In [11]:
%%time
X_test = preprocess_train_test_set(main_path, df_test['path'], image_size)

CPU times: user 2min 1s, sys: 4.54 s, total: 2min 5s
Wall time: 2min 5s


# Cargamos las NNs previamente entrenadas

NN superclases

In [12]:
model_super = keras.models.load_model(path+'NNs/NN_superclases.keras',
                                      custom_objects={'macro_f1': lambda y_true, y_pred: create_macro_f1(y_true, y_pred, 3)})

## NN subclases

In [13]:
with open(path + "hierarchy_dict.json", "r") as archivo:
    hierarchy_dict = json.load(archivo)

with open(path + "food_categories_dict.json", "r") as archivo:
    food_categories_dict = json.load(archivo)

with open(path + "dish_dict.json", "r") as archivo:
    dish_dict = json.load(archivo)

In [14]:
transformed_dishes = transform_dishes(dish_dict, hierarchy_dict, food_categories_dict)

inverse_transformed_dishes = {}

for superclase, dictt in transformed_dishes.items():
  inverse_transformed_dishes[superclase] = {value: key for key, value in dictt.items()}

Superclase 0

In [15]:
superclase = 0
n_subclasses = len(transformed_dishes[str(superclase)])

model_sub_0 = keras.models.load_model(path+f'NNs/NN_subclases_{superclase}.keras',
                                      custom_objects={'macro_f1': lambda y_true, y_pred: create_macro_f1(y_true, y_pred, n_subclasses)})

Superclase 1

In [16]:
superclase = 1
n_subclasses = len(transformed_dishes[str(superclase)])

model_sub_1 = keras.models.load_model(path+f'NNs/NN_subclases_{superclase}.keras',
                                      custom_objects={'macro_f1': lambda y_true, y_pred: create_macro_f1(y_true, y_pred, n_subclasses)})

Superclase 2

In [17]:
superclase = 2
n_subclasses = len(transformed_dishes[str(superclase)])

model_sub_2 = keras.models.load_model(path+f'NNs/NN_subclases_{superclase}.keras',
                                      custom_objects={'macro_f1': lambda y_true, y_pred: create_macro_f1(y_true, y_pred, n_subclasses)})

In [18]:
%%time
super_clases = np.argmax(model_super.predict(X_test, verbose=0), axis=1)

CPU times: user 36 s, sys: 5.99 s, total: 42 s
Wall time: 44.1 s


In [19]:
%%time

y_pred=[]

for i, super_clase in enumerate(super_clases):
  image = np.expand_dims(X_test[i], axis=0) # (225,225,3) -> (1,225,225,3)
  if super_clase == 0:
    sub_clase = np.argmax(model_sub_0.predict(image, verbose=0))
  elif super_clase == 1:
    sub_clase = np.argmax(model_sub_1.predict(image, verbose=0))
  else:
    sub_clase = np.argmax(model_sub_2.predict(image, verbose=0))

  sub_clase = inverse_transformed_dishes[str(super_clase)][sub_clase]

  y_pred.append(f'{super_clase} {sub_clase}')

CPU times: user 26min 41s, sys: 1min 3s, total: 27min 45s
Wall time: 26min 24s


In [20]:
sample_submission = pd.read_csv(main_path+'sample_submission.csv')

In [21]:
sample_submission

,path,classes
0,images/caprese_salad/820512.jpg,0 0
1,images/hamburger/2062556.jpg,0 0
2,images/lasagna/2381454.jpg,0 0
3,images/ceviche/1995.jpg,0 0
4,images/poutine/892377.jpg,0 0
...,...,...
25245,images/ravioli/2087725.jpg,0 0
25246,images/ice_cream/504131.jpg,0 0
25247,images/cannoli/1187751.jpg,0 0
25248,images/pancakes/477236.jpg,0 0


In [22]:
sample_submission['classes'] = y_pred

In [23]:
sample_submission

,path,classes
0,images/caprese_salad/820512.jpg,0 94
1,images/hamburger/2062556.jpg,2 48
2,images/lasagna/2381454.jpg,2 86
3,images/ceviche/1995.jpg,1 50
4,images/poutine/892377.jpg,2 27
...,...,...
25245,images/ravioli/2087725.jpg,1 97
25246,images/ice_cream/504131.jpg,1 55
25247,images/cannoli/1187751.jpg,1 57
25248,images/pancakes/477236.jpg,1 56


In [24]:
sample_submission.to_csv(path+'v3.csv', index=False)